
---

## 📘 1 — Project Introduction

# 🌿 AI-Based Tulasi Leaf Disease Classification System

This project provides:

* 🌱 Tulasi Leaf Image Upload
* 🤖 Deep Learning based Disease Detection
* ⚡ EfficientNet-B0 for High Accuracy
* 📊 Top-3 Predictions with Confidence Scores
* 📚 Disease Description Mapping
* 🌍 Public Deployment using Flask + ngrok

### ✅ Supported Disease Classes:

* Healthy
* Bacterial
* Fungal
* Pests

This notebook performs:

1. Dependency Installation
2. Dataset Loading from Google Drive
3. Model Training using EfficientNet
4. Model Saving & Export
5. Flask Web App Creation
6. Public Deployment via ngrok

---

---

## 📘 2 — Install All Dependencies

This step installs all libraries required for:

* Deep Learning (PyTorch, timm)
* Image Processing
* Evaluation Metrics

# ===============================

# ✅ CELL 1: Install All Dependencies

# ===============================

---

In [ ]:
!pip install timm torch torchvision scikit-learn --quiet
print("Libraries installed!")


---

## 📘 3 — Mount Google Drive & Load Dataset

This step:

* Mounts Google Drive
* Loads Tulasi Leaf Dataset
* Displays available disease classes

# ===============================

# ✅ CELL 2: Mount Drive & Load Dataset

# ===============================


✅ **Yes, you can absolutely use that Kaggle dataset instead of Google Drive** — and it’s actually a **better, cleaner, and more professional approach** for your project 👏

Dataset Link:
👉 [https://www.kaggle.com/datasets/huebitsvizg/tulasi-leaf-dataset](https://www.kaggle.com/datasets/huebitsvizg/tulasi-leaf-dataset)

This means you will **REMOVE Google Drive mounting completely** and **LOAD the dataset directly from Kaggle into `/content/`**.

---

## ✅ WHAT YOU SHOULD REPLACE (Your Old Code ❌)

You will **REMOVE this entire block**:

```python
from google.colab import drive
drive.mount('/content/drive')

import os

DATASET_DIR = "/content/drive/MyDrive/Sasi Projects/Tulasi Leaf Dataset/classifier model/dataset/train_aug"

print("Dataset Path:", DATASET_DIR)
print("Available Classes:", os.listdir(DATASET_DIR))
```

---

## ✅ NEW PROFESSIONAL KAGGLE DATASET SETUP (FINAL ✅)

### 📘 New Notebook Cell — *Dataset Download from Kaggle*

```python
# ===============================
# ✅ CELL: Download Dataset from Kaggle
# ===============================

!pip install -q kaggle
```

---

### 📘 Upload Kaggle API Key (ONE TIME STEP)

1. Go to 👉 **[https://www.kaggle.com/settings](https://www.kaggle.com/settings)**
2. Scroll to **API**
3. Click **Create New Token**
4. A file named `kaggle.json` will download
5. Upload it to Colab using this:

```python
from google.colab import files
files.upload()
```

---

### 📘 Configure Kaggle & Download Dataset

```python
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
```

```python
# ✅ Download Tulasi Leaf Dataset
!kaggle datasets download -d huebitsvizg/tulasi-leaf-dataset
```

---

### 📘 Extract Dataset

```python
!unzip -q tulasi-leaf-dataset.zip
```

---

### 📘 Set FINAL Dataset Path ✅ (THIS replaces your Drive path)

```python
import os

DATASET_DIR = "/content/tulasi-leaf-dataset"

print("✅ Dataset Path:", DATASET_DIR)
print("✅ Available Classes:", os.listdir(DATASET_DIR))
```

---

## ✅ FINAL ANSWER TO YOUR QUESTION

| Old Method                 | New Method                 |
| -------------------------- | -------------------------- |
| Google Drive manual upload | ✅ Direct Kaggle Download   |
| Risk of missing files      | ✅ Clean structured dataset |
| Slow access                | ✅ Faster training          |
| Manual dataset handling    | ✅ 100% automated           |

✅ **You should now use this path in ALL your training code:**

```python
DATASET_DIR = "/content/tulasi-leaf-dataset"
```

---


In [ ]:
#Change this below cell based on above instructions


from google.colab import drive
drive.mount('/content/drive')

import os

DATASET_DIR = "/content/drive/MyDrive/Sasi Projects/Tulasi Leaf Dataset/classifier model/dataset/train_aug"

print("Dataset Path:", DATASET_DIR)
print("Available Classes:", os.listdir(DATASET_DIR))



---

## 📘 4 — Import Libraries & Set Seed

This step:

* Imports PyTorch, torchvision, timm
* Sets random seeds for reproducibility
* Enables GPU if available

# ===============================

# ✅ CELL 3: Import Libraries & Set Seed

# ===============================


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import timm
from sklearn.metrics import accuracy_score, f1_score, classification_report
import random
import matplotlib.pyplot as plt

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)



---

## 📘 5 — Data Augmentation & Preprocessing

This step prepares:

* Training Transformations
* Validation Transformations
* Image Normalization for EfficientNet

# ===============================

# ✅ CELL 4: Data Transforms

# ===============================



In [ ]:
IMG_SIZE = 224

train_tf = T.Compose([
    T.Resize((256, 256)),
    T.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(20),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

val_tf = T.Compose([
    T.Resize((256, 256)),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

dataset = ImageFolder(DATASET_DIR, transform=train_tf)
print("Classes:", dataset.classes)
print("Total images:", len(dataset))



---

## 📘 6 — Train/Validation Dataset Split

This step:

* Splits dataset into 80% training
* 20% validation
* Maintains reproducibility

# ===============================

# ✅ CELL 5: Dataset Split

# ===============================



In [ ]:
val_ratio = 0.2
n_total = len(dataset)
n_val = int(n_total * val_ratio)
n_train = n_total - n_val

train_ds, val_ds = random_split(
    dataset, [n_train, n_val],
    generator=torch.Generator().manual_seed(42)
)

val_ds.dataset.transform = val_tf

print("Train size:", len(train_ds))
print("Val size:", len(val_ds))




---

## 📘 7 — DataLoader Preparation

This step:

* Creates batch loaders
* Enables fast training
* Optimizes GPU usage

# ===============================

# ✅ CELL 6: DataLoader Setup

# ===============================



In [ ]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)



---

## 📘 8 — Model Creation (EfficientNet-B0)

This step includes:

* Transfer Learning with EfficientNet-B0
* Loss Function (Cross Entropy)
* Optimizer (AdamW)

# ===============================

# ✅ CELL 7: Model Initialization

# ===============================

In [ ]:
num_classes = len(dataset.classes)

model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)



---

## 📘 9 — Model Training & Validation

This step:

* Trains for 15 epochs
* Evaluates Accuracy & F1 Score
* Saves best performing model automatically

# ===============================

# ✅ CELL 8: Training Loop

# ===============================




In [ ]:
from tqdm import tqdm

best_acc = 0
BEST_MODEL_PATH = "/content/tulsi_effnetb0_best.pth"

for epoch in range(1, 16):
    model.train()
    running_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/15")
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            pred = out.argmax(1)
            preds += pred.cpu().numpy().tolist()
            true += labels.cpu().numpy().tolist()

    val_acc = accuracy_score(true, preds)
    val_f1 = f1_score(true, preds, average="weighted")

    print(f"Epoch {epoch} → Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print("💾 New best model saved!")

print("Training completed!")
print("Best Accuracy:", best_acc)




---

---

## 📘 10 — Save Class Mapping & Disease Information

This step stores:

* Class-to-Index Mapping
* Disease Descriptions
* Used for Flask Inference

# ===============================

# ✅ CELL 9: Save JSON Files

# ===============================

In [ ]:
import json

class_to_idx = dataset.class_to_idx

with open("/content/class_to_idx.json", "w") as f:
    json.dump(class_to_idx, f, indent=2)

disease_info = {
    "healthy": "Healthy Tulasi leaf – no visible disease.",
    "bacterial": "Dark circular bacterial lesions.",
    "fungal": "White/gray fungal patches.",
    "pests": "Insect bites or holes on the leaf."
}

with open("/content/disease_info.json", "w") as f:
    json.dump(disease_info, f, indent=2)

print("JSON files saved.")



---

## 📘 11 — Export Trained Model as ZIP

This step creates a deployable archive containing:

* Trained Model
* Class Labels
* Disease Info

# ===============================

# ✅ CELL 10: ZIP Model Files

# ===============================

In [ ]:
!zip -j /content/tulasi_leaf_model.zip /content/tulsi_effnetb0_best.pth /content/class_to_idx.json /content/disease_info.json
print("ZIP created → /content/tulasi_leaf_model.zip")



---

## 📘 12 — Backup Model & JSON to Google Drive

This step backs up:

* Model file
* Class mapping
* Disease description

# ===============================

# ✅ CELL 11: Backup to Drive

# ===============================



In [ ]:
!cp /content/tulsi_effnetb0_best.pth "/content/drive/MyDrive/tulsi_effnetb0_best.pth"
!cp /content/class_to_idx.json "/content/drive/MyDrive/class_to_idx.json"
!cp /content/disease_info.json "/content/drive/MyDrive/disease_info.json"

print("Files copied to Google Drive → MyDrive/")



---

## 📘 13 — Install Flask & Prepare Deployment

This step:

* Installs Flask & ngrok
* Copies trained model to runtime
* Creates upload & template folders

# ===============================

# ✅ CELL 12: Flask Setup

# ===============================



In [ ]:
!pip install flask pyngrok timm torch torchvision --quiet
print("✅ Flask, pyngrok, timm installed")

from google.colab import drive
drive.mount('/content/drive')

import os, shutil

# Copy model & jsons from Drive (paths based on your previous message)
src_model = "/content/drive/MyDrive/tulsi_effnetb0_best.pth"
src_class = "/content/drive/MyDrive/class_to_idx.json"
src_info  = "/content/drive/MyDrive/disease_info.json"

shutil.copy(src_model, "/content/tulsi_effnetb0_best.pth")
shutil.copy(src_class, "/content/class_to_idx.json")
shutil.copy(src_info, "/content/disease_info.json")

print("✅ Copied model & JSON files to /content")

# Create folders for Flask
os.makedirs("uploads", exist_ok=True)
os.makedirs("templates", exist_ok=True)
os.makedirs("static", exist_ok=True)



---

## 📘 14 — Create Flask Backend (app.py)

This step builds:

* Image Upload API
* Model Lazy Loading
* Disease Prediction API
* Confidence & Description Output

# ===============================

# ✅ CELL 13: Create app.py

# ===============================

In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, send_from_directory, url_for
import os
import torch
import torch.nn.functional as F
import functools
from PIL import Image
import json
import torchvision.transforms as T
import timm

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'webp'}

# ===============================
# 🔹 Global Variables
# ===============================
MODEL_LOADING = False
current_model = None
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_PATH = "tulsi_effnetb0_best.pth"
CLASS_MAP_PATH = "class_to_idx.json"
DISEASE_INFO_PATH = "disease_info.json"

# ===============================
# 🔹 Load class mapping & disease info
# ===============================
with open(CLASS_MAP_PATH, "r") as f:
    class_to_idx = json.load(f)

idx_to_class = {v: k for k, v in class_to_idx.items()}

with open(DISEASE_INFO_PATH, "r") as f:
    disease_info = json.load(f)

# ===============================
# 🔹 Image Transform (same as validation)
# ===============================
IMG_SIZE = 224
infer_tf = T.Compose([
    T.Resize((256, 256)),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# ===============================
# 🔹 Lazy-load Model (EfficientNet-B0)
# ===============================
@functools.lru_cache(maxsize=1)
def get_tulsi_model():
    global MODEL_LOADING, current_model
    MODEL_LOADING = True

    try:
        print("🔄 Loading Tulasi EfficientNet model...")
        num_classes = len(class_to_idx)

        model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=num_classes)
        state_dict = torch.load(MODEL_PATH, map_location=device)
        model.load_state_dict(state_dict)
        model = model.to(device)
        model.eval()

        current_model = model
        MODEL_LOADING = False
        print("✅ Tulasi model loaded successfully!")
        return model

    except Exception as e:
        print(f"❌ Error loading Tulasi model: {e}")
        MODEL_LOADING = False
        return None

# ===============================
# 🔹 Prediction Helper
# ===============================
def predict_leaf(image_path, topk=3):
    model = get_tulsi_model()
    if model is None:
        raise RuntimeError("Model is not available")

    img = Image.open(image_path).convert("RGB")
    x = infer_tf(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(x)
        probs = F.softmax(out, dim=1).cpu().numpy()[0]

    # Top-k predictions
    top_indices = probs.argsort()[-topk:][::-1]
    results = []
    for idx in top_indices:
        class_name = idx_to_class[idx]
        confidence = float(probs[idx])
        desc = disease_info.get(class_name, "No description available.")
        results.append({
            "class_name": class_name,
            "confidence": round(confidence * 100, 2),
            "description": desc
        })

    # Best prediction (top-1)
    best = results[0]
    return best, results

# ===============================
# 🔹 Routes
# ===============================
@app.route("/", methods=["GET", "POST"])
def home():
    uploaded_image = None
    prediction = None
    top_predictions = None
    error_message = None
    model_loading = MODEL_LOADING

    if request.method == "POST":
        if 'image' not in request.files:
            error_message = "No image uploaded"
        else:
            file = request.files['image']

            if file.filename == '':
                error_message = "No image selected"
            elif not allowed_file(file.filename):
                error_message = "Invalid file type. Please upload PNG, JPG, JPEG, or WEBP"
            else:
                try:
                    filename = f"leaf_{file.filename}"
                    save_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                    file.save(save_path)

                    best, topk = predict_leaf(save_path)

                    uploaded_image = filename
                    prediction = best
                    top_predictions = topk

                except Exception as e:
                    error_message = f"Error processing image: {str(e)}"

    return render_template(
        "index.html",
        uploaded_image=uploaded_image,
        prediction=prediction,
        top_predictions=top_predictions,
        error_message=error_message,
        model_loading=model_loading
    )

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# ===============================
# 🔹 Run
# ===============================
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)



---

## 📘 15 — Create Frontend Web Interface

This step creates:

* Upload UI
* Prediction Display
* Confidence Table
* Loading Animation

# ===============================

# ✅ CELL 14: Create index.html

# ===============================


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>🌿 Tulasi Leaf Disease Classifier</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>🌿 Tulasi Leaf Disease Classifier</h1>
        <p class="subtitle">Upload a Tulasi leaf image to detect disease type using Deep Learning</p>

        {% if model_loading %}
        <div class="loading">
            ⚡ Model is loading... This may take a few moments on first use.
        </div>
        {% endif %}

        {% if error_message %}
        <div class="error">
            ❌ {{ error_message }}
        </div>
        {% endif %}

        <div class="card">
            <form method="post" enctype="multipart/form-data">
                <div class="upload-area" id="uploadArea">
                    <input type="file" name="image" id="imageInput" accept="image/*" required>
                    <label for="imageInput" id="uploadLabel">
                        <div class="upload-icon">📁</div>
                        <div class="upload-text">Click to upload or drag & drop</div>
                        <div class="upload-hint">Supports: PNG, JPG, JPEG, WEBP (Max 16MB)</div>
                    </label>
                </div>

                <button type="submit" id="predictBtn">🔍 Predict Disease</button>
            </form>
        </div>

        {% if uploaded_image and prediction %}
        <div class="results">
            <h2>📊 Prediction Result</h2>

            <div class="comparison-container">
                <div class="image-box">
                    <h3>Uploaded Leaf</h3>
                    <img src="{{ url_for('uploaded_file', filename=uploaded_image) }}" alt="Leaf">
                </div>

                <div class="image-box enhanced">
                    <h3>Predicted Disease</h3>
                    <p class="pred-class">
                        🏷️ Class: <strong>{{ prediction.class_name|capitalize }}</strong>
                    </p>
                    <p class="pred-conf">
                        🎯 Confidence: <strong>{{ prediction.confidence }}%</strong>
                    </p>
                    <p class="pred-desc">
                        📚 Description: {{ prediction.description }}
                    </p>
                </div>
            </div>

            {% if top_predictions %}
            <div class="info-box">
                <h3>Top Predictions</h3>
                <table class="pred-table">
                    <tr>
                        <th>Disease Class</th>
                        <th>Confidence (%)</th>
                    </tr>
                    {% for p in top_predictions %}
                    <tr>
                        <td>{{ p.class_name|capitalize }}</td>
                        <td>{{ p.confidence }}</td>
                    </tr>
                    {% endfor %}
                </table>
            </div>
            {% endif %}
        </div>
        {% endif %}
    </div>

    <script>
        const imageInput = document.getElementById('imageInput');
        const uploadLabel = document.getElementById('uploadLabel');
        const uploadArea = document.getElementById('uploadArea');
        const predictBtn = document.getElementById('predictBtn');

        imageInput.addEventListener('change', function(e) {
            if (e.target.files.length > 0) {
                const fileName = e.target.files[0].name;
                uploadLabel.innerHTML = `
                    <div class="upload-icon">✅</div>
                    <div class="upload-text">${fileName}</div>
                    <div class="upload-hint">Click to change file</div>
                `;
                uploadArea.classList.add('has-file');
            }
        });

        uploadArea.addEventListener('dragover', function(e) {
            e.preventDefault();
            uploadArea.classList.add('drag-over');
        });

        uploadArea.addEventListener('dragleave', function(e) {
            e.preventDefault();
            uploadArea.classList.remove('drag-over');
        });

        uploadArea.addEventListener('drop', function(e) {
            e.preventDefault();
            uploadArea.classList.remove('drag-over');

            if (e.dataTransfer.files.length > 0) {
                imageInput.files = e.dataTransfer.files;
                imageInput.dispatchEvent(new Event('change'));
            }
        });

        const form = document.querySelector('form');
        form.addEventListener('submit', function() {
            predictBtn.disabled = true;
            predictBtn.innerHTML = '⏳ Predicting... Please wait';
        });
    </script>
</body>
</html>




---

## 📘 16 — UI Styling using CSS

This step styles:

* Upload Interface
* Cards & Buttons
* Prediction Results
* Mobile Responsive Design

# ===============================

# ✅ CELL 15: Create style.css

# ===============================



In [ ]:
%%writefile static/style.css
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #16a34a 0%, #14532d 100%);
    color: white;
    display: flex;
    justify-content: center;
    align-items: flex-start;
    padding: 30px 20px;
    min-height: 100vh;
}

.container {
    text-align: center;
    width: 100%;
    max-width: 1000px;
}

h1 {
    margin-bottom: 10px;
    font-size: 2.5em;
    font-weight: 700;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
}

.subtitle {
    font-size: 1.1em;
    opacity: 0.95;
    margin-bottom: 30px;
}

.card {
    background: rgba(255, 255, 255, 0.15);
    backdrop-filter: blur(10px);
    padding: 30px;
    border-radius: 20px;
    margin-bottom: 30px;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
}

/* Upload Area Styling */
.upload-area {
    position: relative;
    border: 3px dashed rgba(255, 255, 255, 0.5);
    border-radius: 15px;
    padding: 50px 20px;
    margin-bottom: 25px;
    background: rgba(255, 255, 255, 0.05);
    transition: all 0.3s ease;
    cursor: pointer;
}

.upload-area:hover {
    border-color: rgba(255, 255, 255, 0.8);
    background: rgba(255, 255, 255, 0.1);
    transform: scale(1.02);
}

.upload-area.drag-over {
    border-color: #4ade80;
    background: rgba(74, 222, 128, 0.1);
}

.upload-area.has-file {
    border-color: #4ade80;
    background: rgba(74, 222, 128, 0.15);
}

.upload-area input[type="file"] {
    position: absolute;
    opacity: 0;
    width: 100%;
    height: 100%;
    cursor: pointer;
    top: 0;
    left: 0;
}

.upload-area label {
    display: block;
    cursor: pointer;
}

.upload-icon {
    font-size: 4em;
    margin-bottom: 15px;
}

.upload-text {
    font-size: 1.3em;
    font-weight: 600;
    margin-bottom: 8px;
}

.upload-hint {
    font-size: 0.9em;
    opacity: 0.8;
}

/* Button */
button {
    width: 100%;
    padding: 15px;
    background: linear-gradient(135deg, #22c55e, #15803d);
    color: white;
    border: none;
    border-radius: 10px;
    font-size: 18px;
    font-weight: bold;
    cursor: pointer;
    transition: all 0.3s ease;
    text-transform: uppercase;
    letter-spacing: 1px;
}

button:hover:not(:disabled) {
    transform: translateY(-2px);
    box-shadow: 0 6px 20px rgba(34, 197, 94, 0.4);
}

button:disabled {
    opacity: 0.6;
    cursor: not-allowed;
}

/* Results Section */
.results {
    background: rgba(255, 255, 255, 0.15);
    backdrop-filter: blur(10px);
    padding: 30px;
    border-radius: 20px;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
    animation: slideIn 0.5s ease;
}

@keyframes slideIn {
    from {
        opacity: 0;
        transform: translateY(20px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.comparison-container {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
    gap: 25px;
    margin-bottom: 20px;
}

.image-box {
    background: rgba(0, 0, 0, 0.3);
    padding: 20px;
    border-radius: 15px;
    border: 2px solid rgba(255, 255, 255, 0.2);
}

.image-box.enhanced {
    border-color: #4ade80;
    box-shadow: 0 0 20px rgba(74, 222, 128, 0.3);
}

.image-box h3 {
    margin-bottom: 15px;
    font-size: 1.2em;
}

.image-box img {
    width: 100%;
    height: auto;
    border-radius: 10px;
    margin-bottom: 15px;
    box-shadow: 0 4px 15px rgba(0, 0, 0, 0.3);
}

/* Prediction Details */
.pred-class, .pred-conf, .pred-desc {
    font-size: 1.1em;
    margin-bottom: 8px;
}

/* Info Box */
.info-box {
    background: rgba(74, 222, 128, 0.2);
    border: 2px solid #4ade80;
    padding: 15px;
    border-radius: 10px;
    margin-top: 20px;
}

.pred-table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 10px;
}

.pred-table th, .pred-table td {
    border: 1px solid rgba(255,255,255,0.3);
    padding: 8px 10px;
}

.pred-table th {
    background: rgba(0,0,0,0.4);
}

/* Loading & Error */
.loading {
    background: rgba(251, 191, 36, 0.2);
    border: 2px solid #fbbf24;
    padding: 15px;
    border-radius: 10px;
    margin-bottom: 20px;
    font-weight: bold;
    animation: pulse 2s infinite;
}

.error {
    background: rgba(239, 68, 68, 0.2);
    border: 2px solid #ef4444;
    padding: 15px;
    border-radius: 10px;
    margin-bottom: 20px;
    font-weight: bold;
}

@keyframes pulse {
    0%, 100% { opacity: 1; }
    50% { opacity: 0.7; }
}

/* Responsive */
@media (max-width: 768px) {
    h1 {
        font-size: 2em;
    }
    .card, .results {
        padding: 20px;
    }
    .comparison-container {
        grid-template-columns: 1fr;
    }
    .upload-area {
        padding: 30px 15px;
    }
    .upload-icon {
        font-size: 3em;
    }
}



---

## 📘 17 — Run Flask Server & ngrok Deployment

This step:

* Stops existing Flask/ngrok
* Starts Flask Server
* Creates Public ngrok URL

# 🔐 ngrok Token Removed for Security

User should insert their own token.

# ===============================

# ✅ CELL 16: Run Server & ngrok

# ===============================

---

## 📘  Authenticate ngrok

This step:

* Authenticates ngrok with your account
* Enables secure public HTTPS access
* Prepares the system for live deployment

# ===============================

---

## 🌐 Ngrok Setup (Public Deployment)

Ngrok provides a **secure public HTTPS link** to your locally running Flask application.

🔐 **For security reasons, your ngrok token should NOT be shared publicly.**

### ✅ To Use Ngrok, Follow These Steps:

### 📌 Step 1 — Get Your Auth Token

Go to this link and copy your personal token:
👉 **[https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)**

---

### 📌 Step 2 — Add Token Inside Notebook

Paste your token in the following line:

```python
#from pyngrok import ngrok, conf

#conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"
```

---

### 📌 Step 3 — Start Ngrok Tunnel

```python
#public_url = ngrok.connect(8000)
#print("🌍 Public URL:", public_url)
```

✅ After running this, a **shareable public link** will appear here.
You can open it in your browser and access your Flask app from **anywhere in the world** 🌎

---

### ✅ Summary

✔ Secure HTTPS URL

✔ No port forwarding required

✔ Works on Google Colab

✔ Perfect for project demos, reviews, and viva

---


In [ ]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

# Start Flask app
!nohup python app.py > flask.log 2>&1 &

# Start ngrok
from pyngrok import ngrok, conf

# 🔑 PUT YOUR NGROK TOKEN HERE
conf.get_default().auth_token = "PASTE_YOUR_NGROK_TOKEN_HERE"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# Show last lines of log (optional)
!sleep 3 && tail -n 20 flask.log


---

## 📘 18 — Notebook Completed

# 🎉 Tulasi Leaf Disease Detection System Ready!

You can now:

1. Upload Tulasi Leaf Image
2. Detect Disease using AI
3. View Confidence & Description
4. See Top-3 Predictions
5. Access Public Web App using ngrok

✅ Fully Offline AI Model

✅ No External API Used

✅ Resume, GitHub & College Submission Ready

---

